In [257]:
import pandas as pd
import numpy as np
import random as rn
import math




In [258]:
data = pd.read_csv('spotify-yt-fixed.csv')


In [259]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x) - 1, 0, -1):
            j = rn.randint(0, i - 1)
            x.iloc[i], x.iloc[j] = x.iloc[j], x.iloc[i]

    @staticmethod
    def normalization(x, names):
        #values = x.select_dtypes(exclude="object")
        columnNames = names
        for column in columnNames:
            data = x.loc[:, column]
            max1 = max(data)
            min1 = min(data)
            for row in range(0, len(x), 1):
                xprim = (x.at[row, column] - min1) / (max1 - min1)
                x.at[row, column] = xprim

    @staticmethod
    def getRating(x):
        lis = [0] * len(x)
        i = 0
        for index, row in x.iterrows():
            lis[i] +=  math.sqrt(row['Views']) + 2 * row['Likes'] / (row['Views']+1) + (row['Comments'] * 0.5) / (row['Views'] +1)
            i+=1
        return lis

    @staticmethod
    def split(x, k):  # k = 0.7, 70% do treningowego
        splitPoint = int(len(x) * k)
        return x.iloc[0:splitPoint], x.iloc[splitPoint:]

    @staticmethod
    def getDistances(x, newObj, columnNames, function):
        lis = [0] * len(x)
        i = 0
        
        for index, row in x.iterrows():
            for column in columnNames:
                lis[i] +=(function(row[column], newObj[column]))
            i+=1
        return lis

    @staticmethod
    def manhattan(first, second):
        return abs(first - second)
                


    @staticmethod
    def sort(x, lis):
        x['distance'] = lis
        return x.sort_values(by=['distance'])





    @staticmethod
    def NaiveBayes(x, sample, classCol):
        classes = x[classCol].unique().tolist()
        collen = len(x.columns) - 1
        res = {}
        for var in classes:
            res[var] = []
            for cl in x.columns.tolist()[:collen]:
                values = x.loc[x[classCol] == var, cl]
                mean = values.mean()
                sigm2 = values.std()**2
                res[var].append(DataProcessing.gauss(sample[cl], mean, sigm2))
            res[var] = 1 / len(classes) * np.prod(res[var])
        return max(res, key=res.get)

    @staticmethod
    def gauss(x, mu, sigm2):
        return (1 / sigm2 * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu) / sigm2)**2)

    @staticmethod
    def useBayes(tS, vS):
        counter = 0
        for i in range(0, len(vS)):
            print(DataProcessing.NaiveBayes(tS, vS.iloc[i], 'variety'), vS.iloc[i]['variety'])
            if DataProcessing.NaiveBayes(tS, vS.iloc[i], 'variety') == vS.iloc[i]['variety']:
                counter += 1
        return counter / len(vS)


In [260]:
# np.random.shuffle(data.values)
data.iloc[0]

Unnamed: 0                       0
Artist                    Gorillaz
Track               Feel Good Inc.
Danceability                 0.818
Energy                       0.705
Key                            6.0
Loudness                    -6.679
Speechiness                  0.177
Acousticness               0.00836
Instrumentalness           0.00233
Liveness                     0.613
Valence                      0.772
Tempo                      138.559
Duration_ms               222640.0
Views                  693555221.0
Likes                    6220896.0
Comments                  169907.0
Name: 0, dtype: object

In [261]:
rating = DataProcessing.getRating(data)
data['Rating'] = rating
DataProcessing.normalization(data,['Loudness', 'Tempo', 'Duration_ms', 'Views', 'Likes', 'Comments', 'Rating'] )
bar = data.sort_values('Rating', ascending=False).loc[1000, 'Rating']
bar


0.18002974158603421

max_row = df['A'].idxmax()

In [262]:
ts, vs = DataProcessing.split(data, 0.9)


dist = DataProcessing.getDistances(vs, vs.iloc[0], ['Danceability', 'Energy', 'Key', 'Loudness', 'Acousticness', 'Instrumentalness',
'Liveness', 'Valence', 'Tempo', 'Duration_ms'], DataProcessing.manhattan)
dist

[0.0,
 1.131873781507574,
 7.685641268084854,
 3.914571288527498,
 7.815322318192513,
 3.344854985782135,
 5.933653738615816,
 2.6260839283922577,
 2.3788991653109592,
 3.300364367555866,
 6.214716200309924,
 4.630515424378139,
 4.349874103585962,
 8.712198543767613,
 6.3282269355980425,
 6.656648442176209,
 2.742917580659442,
 2.0622906895331163,
 8.758820991465278,
 7.618365372520662,
 4.864014092907984,
 7.6086119675961505,
 8.394694679234801,
 9.062329242535926,
 8.642635305158175,
 4.344030042348039,
 4.882726185660623,
 6.171543349086925,
 7.254669089572075,
 4.136076162166836,
 7.375550812589001,
 8.298413703723023,
 7.815322318192513,
 4.338756819757845,
 1.1831417722030133,
 4.143032272294951,
 4.452022045826132,
 2.4134172908446385,
 4.891911836760559,
 5.608114848409399,
 3.1209848601514727,
 2.5410056833670174,
 3.6950789994871824,
 4.814689599746977,
 3.646459817799661,
 7.516634608617929,
 5.303880293703338,
 3.800386192444808,
 8.602142955406926,
 3.0108088384358114,
 4.

In [263]:
vs = DataProcessing.sort(vs, dist)
vs.sort_values('distance')
# vs.sort_values('Rating')

C:\Users\barte\AppData\Local\Temp\ipykernel_16552\2016438381.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['distance'] = lis


,Unnamed: 0,Artist,Track,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Comments,Rating,distance
18089,18663,Christian Nodal,La Mitad,0.724,0.4430,7.0,0.901147,0.0329,0.73000,0.000000,0.101,0.76300,0.399364,0.031605,0.020558,0.019983,8.733992e-04,0.143380,0.000000
18721,19320,Sleepy Hallow,Marie,0.742,0.4980,7.0,0.850735,0.0498,0.70700,0.000000,0.109,0.92600,0.410688,0.032620,0.000355,0.001144,8.704769e-05,0.018843,0.329752
19376,19980,Los Dos Carnales,El Borracho,0.612,0.6720,7.0,0.894172,0.1370,0.68600,0.000000,0.186,0.75100,0.374776,0.032984,0.013433,0.010934,4.978506e-04,0.115903,0.514942
19245,19848,Edgardo Nuñez,Yo Soy El Pirata,0.790,0.5290,7.0,0.807636,0.1210,0.58400,0.000003,0.142,0.67800,0.456848,0.029641,0.001283,0.001113,3.146152e-05,0.035820,0.576961
19935,20553,Eden Muñoz,Me Hace Tanto Bien,0.745,0.4790,7.0,0.859469,0.0265,0.56700,0.000010,0.147,0.67100,0.566433,0.043125,0.004339,0.003160,5.915512e-04,0.065871,0.578277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19768,20386,Miracle Tones,528 Hz Love & Miracles,0.173,0.0683,0.0,0.379682,0.0660,0.93000,0.707000,0.107,0.03720,0.574622,0.022839,0.000014,0.000016,1.243538e-07,0.003762,10.269988
19961,20579,Rain Fruits Sounds,"Rain and Thunderstorm, Pt. 1",0.200,0.9900,1.0,0.472706,0.0687,0.00539,0.921000,0.473,0.00001,0.404570,0.002587,0.000054,0.000097,7.772115e-06,0.007324,10.314265
19661,20278,Solfeggio Healing Frequencies MT,528 Hz Bring Positive Transformation,0.183,0.0270,0.0,0.380870,0.0387,0.96500,0.992000,0.116,0.03700,0.280020,0.017226,0.004211,0.006304,1.258834e-03,0.064889,10.579000
19771,20389,Miracle Tones,528 Hz Bring Positive Transformation,0.183,0.0270,0.0,0.380870,0.0387,0.96500,0.992000,0.116,0.03700,0.280020,0.017226,0.004211,0.006304,1.258834e-03,0.064889,10.579000


In [264]:
first1000 = vs.iloc[:1000]
first1000.sort_values('Rating', ascending=False)


,Unnamed: 0,Artist,Track,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Comments,Rating,distance
19685,20303,CoComelon,Wheels on the Bus,0.941,0.387,9.0,0.727799,0.0427,0.184,0.000029,0.1570,0.9650,0.513703,0.037966,6.063173e-01,2.834657e-01,0.000000e+00,0.778664,3.371078
19690,20308,CoComelon,Baby Shark,0.779,0.587,7.0,0.808124,0.0721,0.330,0.000004,0.0792,0.9080,0.821606,0.020714,3.274720e-01,1.300360e-01,0.000000e+00,0.572252,1.291960
18631,19230,Jhayco,No Me Conoce - Remix,0.804,0.787,10.0,0.899430,0.0746,0.140,0.000000,0.0927,0.5620,0.377915,0.059877,2.449438e-01,1.165723e-01,6.686195e-03,0.494918,4.274739
18630,19229,Jhayco,DÁKITI,0.731,0.573,4.0,0.767251,0.0544,0.401,0.000052,0.1130,0.1450,0.451687,0.037482,1.549941e-01,1.672394e-01,1.647222e-02,0.393693,4.288148
18659,19258,Paulo Londra,Nena Maldición (feat. Lenny Tavárez),0.566,0.531,6.0,0.839838,0.0716,0.192,0.000000,0.1790,0.3670,0.488799,0.042310,1.449304e-01,1.255456e-01,1.556804e-02,0.380697,2.419449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19217,19820,Tems,Replay,0.613,0.549,9.0,0.806936,0.2310,0.097,0.000000,0.1160,0.7800,0.509011,0.032080,4.876449e-07,1.555466e-06,3.108846e-07,0.000699,3.086333
18249,18833,Clairo,Amoeba,0.739,0.491,6.0,0.736385,0.0270,0.791,0.036600,0.1000,0.7070,0.456125,0.042612,4.118991e-07,3.603167e-06,4.974154e-07,0.000643,1.450131
19673,20291,Kato,Howl's Moving Castle Lofi (Merry Go Round),0.767,0.208,6.0,0.520744,0.0601,0.533,0.857000,0.0980,0.2320,0.493438,0.006903,2.105289e-07,8.860247e-07,1.243538e-07,0.000459,3.365179
19930,20548,FIFA Sound,FREE - FIFAe Theme - DANO Remix,0.714,0.549,10.0,0.725552,0.2630,0.321,0.000000,0.2580,0.7760,0.574688,0.026321,1.601555e-07,1.772049e-07,0.000000e+00,0.000400,4.051203
